# Cloud service Configs
> Cloud provider-specific logging configuration classes.

In [1]:
#| default_exp configs.cloud

In [2]:
#| export
from dc_logger.client.base import OutputMode, LogLevel, ServiceConfig

from dc_logger.client.exceptions import LogConfigError

In [3]:
#| exporti

import os
from typing import Optional, Dict, Any
from dataclasses import dataclass, field
from abc import abstractmethod

In [4]:
#| export

@dataclass
class CloudServiceConfig(ServiceConfig):
    cloud_provider : str

    @abstractmethod
    def to_platform_config(self) -> Dict[str, Any]:
        """Get cloud provider specific configuration"""
        raise NotImplementedError()

In [5]:
#| export

@dataclass
class Datadog_ServiceConfig(CloudServiceConfig):
    """Datadog-specific log configuration"""
    output_mode: OutputMode = "cloud"
    cloud_provider: str = "datadog"
    
    api_key: Optional[str] = field(default=None, repr=False)
    app_key: Optional[str] = field(default=None, repr=False)
    
    site: str = "datadoghq.com"
    service: str = "dc_logger"
    env: str = "production"


    def to_platform_config(self) -> Dict[str, Any]:
        return {
            "api_key": self.api_key,
            "app_key": self.app_key,
            "site": self.site,
            "service": self.service,
            "env": self.env,
            "cloud_provider": self.cloud_provider,
        }
    
    def validate_config(self) -> bool:
        if not self.api_key:
            raise LogConfigError("Datadog API key is required")
        return True


In [10]:
dds = Datadog_ServiceConfig(
    api_key = 123
)


In [7]:
#| export


@dataclass
class AWSCloudWatch_ServiceConfig(CloudServiceConfig):
    """AWS CloudWatch-specific log configuration"""
    output_mode: OutputMode = "cloud"
    cloud_provider: str = "aws"
    
    aws_region: str = ""
    log_group: str = ""

    log_stream: Optional[str] = None

    def to_platform_config(self) -> Dict[str, Any]:
        return {
            "aws_region": self.aws_region,
            "log_group": self.log_group,
            "log_stream": self.log_stream,
            "cloud_provider": self.cloud_provider,
        }
    
    def validate_config(self) -> bool:
        if not self.aws_region:
            raise LogConfigError("AWS region is required")
        if not self.log_group:
            raise LogConfigError("AWS log group is required")
        return True

@dataclass
class GCPLogging_ServiceConfig(CloudServiceConfig):
    """Google Cloud Logging-specific configuration"""
    
    output_mode: OutputMode = "cloud"
    cloud_provider: str = "gcp"

    log_name: str = "dc_logger"
    project_id: Optional[str] = None

    def to_platform_config(self) -> Dict[str, Any]:
        return {
            "project_id": self.project_id,
            "log_name": self.log_name,
            "cloud_provider": self.cloud_provider,
        }
    
    def validate_config(self) -> bool:
        if not self.project_id:
            raise LogConfigError("GCP project ID is required")
        return True


@dataclass
class AzureLogAnalytics_ServiceConfig(CloudServiceConfig):
    """Azure Log Analytics-specific configuration"""
    
    workspace_id: Optional[str] = field(default= None)
    shared_key: Optional[str] = field(default = None)
    
    log_type: str = "dc_logger"

    output_mode: str = field(default="cloud", init=False)
    cloud_provider: str = field(default="azure", init=False)
    
    def to_platform_config(self) -> Dict[str, Any]:
        return {
            "workspace_id": self.workspace_id,
            "shared_key": self.shared_key,
            "log_type": self.log_type,
            "cloud_provider": self.cloud_provider,
        }
    def validate_config(self) -> bool:
        if not self.workspace_id:
            raise LogConfigError("Azure workspace ID is required")
        if not self.shared_key:
            raise LogConfigError("Azure shared key is required")
        return True

In [8]:
#| hide
import nbdev; nbdev.nbdev_export('./cloud.ipynb')